# N-Ary Trees

## Background

In [4]:
from typing import Optional, List
from collections import deque

## Traversals
- No standard for traversal orders, but will be generalized below

### Pre-order Traversal
- Visit the root node first, then traverse the subtree rooted at its children one by one

In [1]:
def preorder(root):
    if root is None:
        return []

    stack, output = [root], []  
    
    while stack:
        root = stack.pop()
        
        output.append(root.val)
        
        stack.extend(root.children[::-1])

    return output

### Post-order Traversal
- Traverse the subtree rooted at its children first, then visit the root node

In [3]:
def postorder(root: "Node") -> List[int]:
    result = []

    if root is None:
        return result

    node_stack = [(root, False)]

    while node_stack:
        current_node, is_visited = node_stack.pop()

        if is_visited:
            result.append(current_node.val)
        else:
            node_stack.append((current_node, True))

            for child in reversed(current_node.children):
                node_stack.append((child, False))

    return result

### Level-order Traversal

In [5]:
def levelOrder(root: Optional['Node']) -> List[List[int]]:
    result = []
    
    if root is None:
        return result
    
    queue = deque([root])
    
    while queue:
        level = []
        
        for _ in range(len(queue)):
            node = queue.popleft()
            
            level.append(node.val)
            
            queue.extend(node.children)
        
        result.append(level)
        
    return result

## Problems (Study These)
### Encode N-ary Tree to Binary Tree

In [ ]:
class Node(object):
    def __init__(self, val=None, children=None):
        self.val = val
        self.children = children

class TreeNode(object):
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Codec:
    def encode(self, root):
        """Encodes an n-ary tree to a binary tree.
        :type root: Node
        :rtype: TreeNode
        """
        if not root:
            return None

        rootNode = TreeNode(root.val)
        
        if len(root.children) > 0:
            firstChild = root.children[0]
            rootNode.left = self.encode(firstChild)

        # the parent for the rest of the children
        curr = rootNode.left

        # encode the rest of the children
        for i in range(1, len(root.children)):
            curr.right = self.encode(root.children[i])
            curr = curr.right

        return rootNode


    def decode(self, data):
        """Decodes your binary tree to an n-ary tree.
        :type data: TreeNode
        :rtype: Node
        """
        if not data:
            return None

        rootNode = Node(data.val, [])

        curr = data.left
        
        while curr:
            rootNode.children.append(self.decode(curr))
            curr = curr.right

        return rootNode

### Serialize and Deserialize N-ary Tree

In [ ]:
import collections 

class Codec:

    def _serializeHelper(self, root, serializedList):
        queue = collections.deque() 
        
        queue.append(root)
        queue.append(None)
        
        while queue:
            # Pop a node
            node = queue.popleft()
            
            # If this is an "endNode", we need to add another one
            # to mark the end of the current level unless this
            # was the last level.
            if (node == None):
                # We add a sentinal value of "#" here
                serializedList.append('#');
                if queue:
                    queue.append(None);  
                    
            elif node == 'C':
                # Add a sentinal value of "$" here to mark the switch to a
                # different parent.
                serializedList.append('$');
            else:
                # Add value of the current node and add all of it's
                # children nodes to the queue. Note how we convert
                # the integers to their corresponding ASCII counterparts.
                serializedList.append(chr(node.val + 48));
                for child in node.children:
                    queue.append(child);
                
                # If this not is NOT the last one on the current level, 
                # add a childNode as well since we move on to processing
                # the next node.
                if queue[0] != None:
                    queue.append('C');
        
    def serialize(self, root: 'Node') -> str:
        """Encodes a tree to a single string.
        
        :type root: Node
        :rtype: str
        """
        
        if not root:
            return ""
        
        serializedList = []
        self._serializeHelper(root, serializedList)
        
        return "".join(serializedList)
        
    def _deserializeHelper(self, data, rootNode):
        # We move one level at a time and at every level, we need access
        # to the nodes on the previous level as well so that we can form
        # the children arrays properly. Hence two arrays.
        prevLevel, currentLevel = collections.deque(), collections.deque()
        currentLevel.append(rootNode);
        parentNode = rootNode;
        
        # Process the characters in the string one at a time.
        for i in range (1, len(data)):
            if data[i] == '#':
                
                # Special processing for end of level. We need to swap the
                # array lists. Here, we simply re-initialize the "currentLevel"
                # arraylist rather than clearing it.
                prevLevel = currentLevel;
                currentLevel = collections.deque()
                
                # Since we move one level down, we take the parent as the first
                # node on the current level.
                parentNode = prevLevel.popleft() if prevLevel else None
            else:
                if data[i] == '$':
                    
                    # Special handling for change in parent on the same level
                    parentNode = prevLevel.popleft() if prevLevel else None
                else:
                    childNode = Node(ord(data[i]) - 48, [])
                    
                    currentLevel.append(childNode)
                    parentNode.children.append(childNode)
                   
    def deserialize(self, data: str) -> 'Node':
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: Node
        """
        if not data:
            return None
        
        rootNode = Node(ord(data[0]) - 48, [])
        self._deserializeHelper(data, rootNode)
        
        return rootNode